#코랩 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#라이브러리

In [90]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

import seaborn as sns
import warnings
from itertools import combinations
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier



#모델1

In [91]:
train  = pd.read_csv('/content/drive/My Drive/csv/sw/train.csv')
test  = pd.read_csv('/content/drive/My Drive/csv/sw/test.csv')

In [92]:
x_train = train.copy()
x_train.drop('nerdiness', axis=1, inplace = True)
y_train = train['nerdiness'] 

In [93]:
dataset = [x_train, test]

#이상치 및 결측치 처리

In [94]:
for data in dataset:
  data = data.drop(data[data.introelapse > 15000].index)
  data = data.drop(data[data.testelapse > 100000].index)
  data = data.drop(data[data.surveyelapse > 1000000].index)

In [95]:
Answers1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20',
             'Q21','Q22','Q23','Q24','Q25',
             'Q26']

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
for data in dataset:
  data[Answers1] = imputer.fit_transform(data[Answers1])

In [96]:
Answers3 = ['VCL1','VCL2','VCL3','VCL4','VCL5',
            'VCL6','VCL7','VCL8','VCL9','VCL10',
            'VCL11','VCL12','VCL13','VCL14','VCL15',
            'VCL16']

#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무) 
#-> education 결측값 처리
for data in dataset:
  data['VLC_score'] = data[Answers3].sum(axis=1)       
  data['VLC_var'] = data[Answers3].var(axis=1)
  #data['VLC_mean'] = data[Answers3].mean(axis = 1)   

In [97]:
#country, married, ASD, hand (+voted, orientation) 피처의 최빈값으로 결측치 채운다
for data in dataset:
  data['country'] = data['country'].fillna(data['country'].mode()[0])
  data['married'] = data['married'].fillna(data['married'].mode()[0])
  data['ASD'] = data['ASD'].fillna(data['ASD'].mode()[0])
  data['hand'] = data['hand'].fillna(data['hand'].mode()[0])
  data['voted'] = data['voted'].fillna(data['voted'].mode()[0])
  data['orientation'] = data['orientation'].fillna(data['orientation'].mode()[0])
  #religion은 평균값으로 채운다
  data['religion'] = data['religion'].fillna(data['religion'].mean())


In [98]:
#젠더 3(무응답 예상 값)을 male과 female 값의 중간값인 1.5 로 채우고, 결측치 또한 중간값으로 채운다
for data in dataset:
  data['gender'] = data['gender'].fillna(1.5)
  data_temp_df = data[data['gender'] == 3.0].copy()
  data_temp_df['gender'] = 1.5
  data[data['gender'] == 3.0] = data_temp_df.copy()


In [99]:
#Familysize 12이상은 12로, 결측치는 평균값으로
for data in dataset:
  data_temp_df = data[data['familysize'] >= 12].copy()
  data_temp_df['familysize'] = 12
  data[data['familysize'] >= 12] = data_temp_df.copy()
  data['familysize'] = data['familysize'].fillna(data['familysize'].mean())


In [100]:
#education 비율에 맞춰 VLC 백분위수의 값에 따라 각각의 교육 수준으로 채움
#(VLC 결측값이 존재하지 않아서 가능)
for data in dataset:
  edu_1_count = data['education'].value_counts().get(1)
  edu_2_count = data['education'].value_counts().get(2)
  edu_3_count = data['education'].value_counts().get(3)

  total = edu_1_count+edu_2_count+edu_3_count+data['education'].value_counts().get(4)
  edu_1 = round(((edu_1_count)/total)*100,2)/100
  edu_2 = round(((edu_1_count+edu_2_count)/total)*100,2)/100
  edu_3 = round(((edu_1_count+edu_2_count+edu_3_count)/total)*100,2)/100

  data_temp_df = data[data['VLC_score'] <= data['VLC_score'].quantile(edu_1)].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(1)
  data[data['VLC_score'] <= data['VLC_score'].quantile(edu_1)] = data_temp_df.copy()

  data_temp_df = data[(data['VLC_score'] > data['VLC_score'].quantile(edu_1)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_2))].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(2)
  data[(data['VLC_score'] > data['VLC_score'].quantile(edu_1)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_2))] = data_temp_df.copy()

  data_temp_df = data[(data['VLC_score'] > data['VLC_score'].quantile(edu_2)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_3))].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(3)
  data[(data['VLC_score'] > data['VLC_score'].quantile(edu_2)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_3))] = data_temp_df.copy()

  data_temp_df = data[data['VLC_score'] > data['VLC_score'].quantile(edu_3)].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(4)
  data[data['VLC_score'] > data['VLC_score'].quantile(edu_3)] = data_temp_df.copy()


In [101]:
#USA, GBR, CAN, AUS일 경우 engnat 1 나머지는 2
#USA 7609명중 6893명 영어, GBR 1109명중 978명 영어, CAN 915명중 737명 영어, AUS 525명중 470명 영어
for data in dataset:
  data_temp_df = data[(data['country']=='USA') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='USA') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='GBR') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='GBR') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='CAN') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='CAN') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='AUS') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='AUS') & (data['engnat'].isna())] = data_temp_df.copy()

  data['engnat'] = data['engnat'].fillna(2)

In [102]:
Answers2 = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5',
            'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']

from sklearn.impute import KNNImputer
#회의에서 언급된 knn을 이용해 결측치를 변경
#훈련셋에서 발견된 n_neighbors 가장 가까운 이웃 의 평균값을 사용하여 대치

imputer = KNNImputer(n_neighbors=5)
for data in dataset:
  data[Answers2] = imputer.fit_transform(data[Answers2])


#피처 엔지니어링

In [103]:
for data in dataset:
  data['T'] = data['Q1'] + data['Q2'] - data['Q3'] + data['Q15']
  data['V'] = data['Q5'] - data['Q4'] + data['Q7'] + data['Q8'] - data['Q10'] - data['Q11'] + data['Q12']  + data['Q13'] - data['Q17'] + data['Q20']
  data['M'] = data['Q19'] - data['Q9']
  #마이너스 부호 뒤집기는 안 함- 히트맵으로 본 결과 뒤집으면 음 상관관계를 띄기 때문

In [104]:
#Q_ 질문에 대한 답을 평균낸 Mach_score columns 생성
for data in dataset:
  data['Mach_score'] = data[Answers1].mean(axis = 1)
#분산
for data in dataset:
  data['mach_var'] = data[Answers1].var(axis=1)
#조합 나누기
#Ancoms = list(combinations(Answers1, 2))
#for a,b in Ancoms:
#    data['machQ_%s_dv_%s'%(a,b)] = data[a]/data[b]

In [105]:
data['introelapse'] = data['introelapse'] ** (1/10) #질문들에 소요된 시간
data['testelapse'] = data['testelapse'] ** (1/10) #질문들에 소요된 시간
data['surveyelapse'] = data['surveyelapse'] ** (1/10) #나머지 설문들에 소요된 시간


#data['introelapse_delay_var'] = data['introelapse'].var()
#data['testelapse_delay_var'] = data['testelapse'].var()
#data['surveyelapse_delay_var'] = data['surveyelapse'].var()

In [106]:
#뒤집기
tps = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']
for data in dataset:
  for tp in tps:
    data[tp] = 6 - data[tp]

In [109]:
data['Ex'] = (data['TIPI1']+data['TIPI6'])/2
data['Ag'] = (data['TIPI7']+data['TIPI2'])/2
data['Con'] = (data['TIPI3']+data['TIPI8'])/2
data['Es'] =(data['TIPI9']+data['TIPI4'])/2
data['Op'] =(data['TIPI5']+data['TIPI10'])/2

In [110]:
#EDA 결과, 성별에 따라 Emotional Stability/ Conscience/ Open Minded가 투표 여부에 미치는 영향이 크다고 판단되어 feature를 추가해주었습니다.
#(심리성향 1위 코드)
for data in dataset:
  data['Es_gender'] = data['Es']*data['gender']
  data['Con_gender'] = data['Con']*data['gender']
  data['Op_gender'] = data['Op']*data['gender']

In [111]:
tpcoms = list(combinations(tps, 2))
for data in dataset:
  for a,b in tpcoms:
    data['tp_%s_dv_%s'%(a,b)] = data[a]/data[b]

In [112]:
for data in dataset:
#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무)
  data['VLC_score'] = data[Answers3].sum(axis=1)       
  data['VLC_var'] = data[Answers3].var(axis=1)
  data['VLC_mean'] = data[Answers3].mean(axis = 1)    

In [113]:
#age로 age_group object 생성
def age_grouping(age):
    age = (age//10.)*10
    if(age >= 70):
        age = '+70s'
    else:
        age = str(int(age))+'s'
    return age
for data in dataset:
  data['age_group'] = data['age'].apply(age_grouping)
  data.drop('age', axis=1, inplace = True)


In [114]:
#라벨링 ex) 라벨 인코더
from sklearn.preprocessing import LabelEncoder
for data in dataset:
  encoder = LabelEncoder()
  data['age_group'] = encoder.fit_transform(data['age_group'])

In [115]:
for data in dataset:

  #결측값는 'others'로 대체, 위에서 최빈값으로 대체했다면, 실행할 필요x
  data['country']=data['country'].fillna(1)
  data.country = data.country.apply(lambda x : 'others' if x == 1 else x)

  #1번 밖에 입력되지않은 나라는 'others'로
  country_vc = data['country'].value_counts()
  data.country = data.country.apply(lambda x : 'others' if country_vc[x] == 1 else x)
  
print(x_train['country'].value_counts())

USA    7609
GBR    1109
CAN     915
AUS     525
DEU     473
       ... 
ZMB       2
FRO       2
BWA       2
CUW       2
PAN       2
Name: country, Length: 110, dtype: int64


In [116]:
for data in dataset:
  data = pd.get_dummies(data, columns = ['country'])
  

In [117]:
for data in dataset:
  data.drop('country', axis=1, inplace = True)

In [118]:
#왼손 오른손은 상관없는 피처라고 판단후 삭제
for data in dataset:
  data.drop('hand', axis=1, inplace = True)


#모델1 실행

In [119]:
#k_fold 분할, n_splits = 3 => 훈련, 검증 3개 만든다.
k_fold = KFold(n_splits = 3, shuffle = True, random_state = 0)
# 모델 3개 앙상블 (소프트 보팅)
clf1 = RandomForestClassifier(n_estimators=500)
clf2 = LGBMClassifier()
clf3 = GradientBoostingClassifier()
soft_vote  = VotingClassifier([('r1',clf1), ('r2', clf2), ('r3',clf3)], voting='soft')
soft_vote.fit(x_train, y_train)

VotingClassifier(estimators=[('r1', RandomForestClassifier(n_estimators=500)),
                             ('r2', LGBMClassifier()),
                             ('r3', GradientBoostingClassifier())],
                 voting='soft')

In [120]:
from sklearn import *

model = soft_vote
pred_y = model.predict_proba(test) #test 데이터 예측 확률
pred_y = pred_y[:,1] #[:,1]로 voted가 2일 확률값(투표를 안한 확률 수) 추출

submission = pd.DataFrame()
submission['index'] = submission.index
submission['nerdiness'] = pred_y
submission['index'] = submission.index

submission.to_csv('/content/drive/My Drive/csv/sw/model1.csv', index=False)


#모델2

#data load

In [121]:
train  = pd.read_csv('/content/drive/My Drive/csv/sw/train.csv')
test  = pd.read_csv('/content/drive/My Drive/csv/sw/test.csv')

#EDA

In [122]:
# train 데이터 준비, 
x_train = train.copy()
x_train.drop('nerdiness', axis=1, inplace = True) 
y_train = train['nerdiness'] 

In [123]:
dataset = [x_train, test]

시간 이상치 제거

In [124]:
#시간 데이터 확인
for data in dataset:
  print(data['introelapse'][data.introelapse > 15000])
  print(data['testelapse'][data.testelapse > 100000])
  print(data['surveyelapse'][data.surveyelapse > 1000000])

96       133690
113       27490
277       36976
373      285418
394      258417
          ...  
14201     36976
14335     92602
14488     82944
14665    817147
14940     19723
Name: introelapse, Length: 96, dtype: int64
373      285399
1779     285399
3580     127962
3698     407208
3963     240022
5196     474572
6836     474572
11424    240022
12427    374744
Name: testelapse, dtype: int64
1702     15166994
4367     15166994
13517     3420008
14906     2513542
Name: surveyelapse, dtype: int64
102       89155
214       58249
343       36428
1039      78488
1214      21594
          ...  
35047     39423
35152    132160
35246    246807
35272     89798
35429    101167
Name: introelapse, Length: 218, dtype: int64
1908     257638
2003     679872
3459     257638
3545     127962
4077     434588
4536     164867
8146     121771
8774     164867
10967    312045
12775    434588
15025    698533
15143    407208
16768    121771
21459    312045
26956    311476
27225    698533
28298    311476
28387  

In [125]:
#이상치 제거
for data in dataset:
  data = data.drop(data[data.introelapse > 15000].index)
  data = data.drop(data[data.testelapse > 100000].index)
  data = data.drop(data[data.surveyelapse > 1000000].index)

#결측값 변경

In [126]:
#결측치 변경되었는지 확인하기 위해 결측치 가지는 행 일부 저장
before = x_train.loc[[207,384,1085,5382,7533,9031],:]

In [127]:
Answers1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20',
             'Q21','Q22','Q23','Q24','Q25',
             'Q26']

from sklearn.impute import KNNImputer
#회의에서 언급된 knn을 이용해 결측치를 변경
#훈련셋에서 발견된 n_neighbors 가장 가까운 이웃 의 평균값을 사용하여 대치

imputer = KNNImputer(n_neighbors=5)
for data in dataset:
  data[Answers1] = imputer.fit_transform(data[Answers1])

In [128]:
before.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
207,207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,14,1.0,2.0,5.0,2.0,1.0,5.0,2.0
384,384,NaN,4.0,4.0,1.0,5.0,3.0,5.0,1.0,4.0,...,2.0,1.0,46,1.0,12.0,1.0,2.0,1.0,2.0,2.0
1085,1085,NaN,5.0,5.0,5.0,5.0,NaN,5.0,5.0,5.0,...,1.0,2.0,25,1.0,1.0,1.0,1.0,1.0,2.0,1.0
5382,5382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,15,1.0,7.0,2.0,2.0,1.0,2.0,2.0
7533,7533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,15,1.0,2.0,1.0,2.0,1.0,3.0,2.0


In [129]:
#결측값이 어떤 식으로 대체 되었는지 비교해서 확인
after = x_train.loc[[207,384,1085,5382,7533,9031],:]
after.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
207,207,3.972859,4.065702,4.222809,3.795499,3.856503,3.676364,4.183262,3.879013,3.915418,...,2.0,1.0,14,1.0,2.0,5.0,2.0,1.0,5.0,2.0
384,384,3.800000,4.000000,4.000000,1.000000,5.000000,3.000000,5.000000,1.000000,4.000000,...,2.0,1.0,46,1.0,12.0,1.0,2.0,1.0,2.0,2.0
1085,1085,4.800000,5.000000,5.000000,5.000000,5.000000,3.800000,5.000000,5.000000,5.000000,...,1.0,2.0,25,1.0,1.0,1.0,1.0,1.0,2.0,1.0
5382,5382,3.972859,4.065702,4.222809,3.795499,3.856503,3.676364,4.183262,3.879013,3.915418,...,1.0,1.0,15,1.0,7.0,2.0,2.0,1.0,2.0,2.0
7533,7533,3.972859,4.065702,4.222809,3.795499,3.856503,3.676364,4.183262,3.879013,3.915418,...,1.0,2.0,15,1.0,2.0,1.0,2.0,1.0,3.0,2.0


In [130]:
# Q_의 결측치가 0 
for data in dataset:
  print('data : ',data.isnull().sum())

data :  index            0
Q1               0
Q2               0
Q3               0
Q4               0
              ... 
orientation    399
voted           85
married         82
familysize     319
ASD             89
Length: 69, dtype: int64
data :  index            0
Q1               0
Q2               0
Q3               0
Q4               0
              ... 
orientation    823
voted          181
married        180
familysize     703
ASD            237
Length: 69, dtype: int64


In [131]:
Answers3 = ['VCL1','VCL2','VCL3','VCL4','VCL5',
            'VCL6','VCL7','VCL8','VCL9','VCL10',
            'VCL11','VCL12','VCL13','VCL14','VCL15',
            'VCL16']

#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무) 
#-> education 결측값 처리
for data in dataset:
  data['VLC_score'] = data[Answers3].sum(axis=1)       
  data['VLC_var'] = data[Answers3].var(axis=1)
  data['VLC_mean'] = data[Answers3].mean(axis = 1)   

In [132]:
#country, married, ASD, hand (+voted, orientation) 피처의 최빈값으로 결측치 채운다
for data in dataset:
  data['country'] = data['country'].fillna(data['country'].mode()[0])
  data['married'] = data['married'].fillna(data['married'].mode()[0])
  data['ASD'] = data['ASD'].fillna(data['ASD'].mode()[0])
  data['hand'] = data['hand'].fillna(data['hand'].mode()[0])
  data['voted'] = data['voted'].fillna(data['voted'].mode()[0])
  data['orientation'] = data['orientation'].fillna(data['orientation'].mode()[0])
  #religion은 평균값으로 채운다
  data['religion'] = data['religion'].fillna(data['religion'].mean())


In [133]:
#젠더 3(무응답 예상 값)을 male과 female 값의 중간값인 1.5 로 채우고, 결측치 또한 중간값으로 채운다
for data in dataset:
  data['gender'] = data['gender'].fillna(1.5)
  data_temp_df = data[data['gender'] == 3.0].copy()
  data_temp_df['gender'] = 1.5
  data[data['gender'] == 3.0] = data_temp_df.copy()


In [134]:
#Familysize 12이상은 12로, 결측치는 평균값으로
for data in dataset:
  data_temp_df = data[data['familysize'] >= 12].copy()
  data_temp_df['familysize'] = 12
  data[data['familysize'] >= 12] = data_temp_df.copy()
  data['familysize'] = data['familysize'].fillna(data['familysize'].mean())


In [135]:
#education 비율에 맞춰 VLC 백분위수의 값에 따라 각각의 교육 수준으로 채움
#(VLC 결측값이 존재하지 않아서 가능)
for data in dataset:
  edu_1_count = data['education'].value_counts().get(1)
  edu_2_count = data['education'].value_counts().get(2)
  edu_3_count = data['education'].value_counts().get(3)

  total = edu_1_count+edu_2_count+edu_3_count+data['education'].value_counts().get(4)
  edu_1 = round(((edu_1_count)/total)*100,2)/100
  edu_2 = round(((edu_1_count+edu_2_count)/total)*100,2)/100
  edu_3 = round(((edu_1_count+edu_2_count+edu_3_count)/total)*100,2)/100

  data_temp_df = data[data['VLC_score'] <= data['VLC_score'].quantile(edu_1)].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(1)
  data[data['VLC_score'] <= data['VLC_score'].quantile(edu_1)] = data_temp_df.copy()

  data_temp_df = data[(data['VLC_score'] > data['VLC_score'].quantile(edu_1)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_2))].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(2)
  data[(data['VLC_score'] > data['VLC_score'].quantile(edu_1)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_2))] = data_temp_df.copy()

  data_temp_df = data[(data['VLC_score'] > data['VLC_score'].quantile(edu_2)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_3))].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(3)
  data[(data['VLC_score'] > data['VLC_score'].quantile(edu_2)) & (data['VLC_score'] <= data['VLC_score'].quantile(edu_3))] = data_temp_df.copy()

  data_temp_df = data[data['VLC_score'] > data['VLC_score'].quantile(edu_3)].copy()
  data_temp_df['education'] = data_temp_df['education'].fillna(4)
  data[data['VLC_score'] > data['VLC_score'].quantile(edu_3)] = data_temp_df.copy()


In [136]:
#USA, GBR, CAN, AUS일 경우 engnat 1 나머지는 2
#USA 7609명중 6893명 영어, GBR 1109명중 978명 영어, CAN 915명중 737명 영어, AUS 525명중 470명 영어
for data in dataset:
  data_temp_df = data[(data['country']=='USA') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='USA') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='GBR') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='GBR') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='CAN') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='CAN') & (data['engnat'].isna())] = data_temp_df.copy()

  data_temp_df = data[(data['country']=='AUS') & (data['engnat'].isna())].copy()
  data_temp_df['engnat'] = data_temp_df['engnat'].fillna(1)
  data[(data['country']=='AUS') & (data['engnat'].isna())] = data_temp_df.copy()

  data['engnat'] = data['engnat'].fillna(2)

In [137]:
Answers2 = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5',
            'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']

from sklearn.impute import KNNImputer
#회의에서 언급된 knn을 이용해 결측치를 변경
#훈련셋에서 발견된 n_neighbors 가장 가까운 이웃 의 평균값을 사용하여 대치

imputer = KNNImputer(n_neighbors=5)
for data in dataset:
  data[Answers2] = imputer.fit_transform(data[Answers2])


In [138]:
for data in dataset:
  print('data : ',data.isnull().sum())

data :  index         0
Q1            0
Q2            0
Q3            0
Q4            0
             ..
familysize    0
ASD           0
VLC_score     0
VLC_var       0
VLC_mean      0
Length: 72, dtype: int64
data :  index         0
Q1            0
Q2            0
Q3            0
Q4            0
             ..
familysize    0
ASD           0
VLC_score     0
VLC_var       0
VLC_mean      0
Length: 72, dtype: int64


#피처 엔지니어링

Q_ 데이터

In [139]:
for data in dataset:
  data['T'] = data['Q1'] + data['Q2'] - data['Q3'] + data['Q15']
  data['V'] = data['Q5'] - data['Q4'] + data['Q7'] + data['Q8'] - data['Q10'] - data['Q11'] + data['Q12']  + data['Q13'] - data['Q17'] + data['Q20']
  data['M'] = data['Q19'] - data['Q9']
  #마이너스 부호 뒤집기는 안 함- 히트맵으로 본 결과 뒤집으면 음 상관관계를 띄기 때문

In [140]:
#Q_ 질문에 대한 답을 평균낸 Mach_score columns 생성
for data in dataset:
  data['Mach_score'] = data[Answers1].mean(axis = 1)
#분산
for data in dataset:
  data['mach_var'] = data[Answers1].var(axis=1)
#조합 나누기
Ancoms = list(combinations(Answers1, 2))
for a,b in Ancoms:
    data['machQ_%s_dv_%s'%(a,b)] = data[a]/data[b]

시간 데이터

In [141]:
data['introelapse_delay'] = data['introelapse'] ** (1/10) #질문들에 소요된 시간
data['testelapse_delay'] = data['testelapse'] ** (1/10) #질문들에 소요된 시간
data['surveyelapse_delay'] = data['surveyelapse'] ** (1/10) #나머지 설문들에 소요된 시간
data['introelapse_delay_var'] = data['introelapse'].var()
data['testelapse_delay_var'] = data['testelapse'].var()
data['surveyelapse_delay_var'] = data['surveyelapse'].var()

TIPI

In [142]:
#뒤집기
tps = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']
for data in dataset:
  for tp in tps:
    data[tp] = 6 - data[tp]

In [145]:
data['Ex'] = (data['TIPI1']+data['TIPI6'])/2
data['Ag'] = (data['TIPI7']+data['TIPI2'])/2
data['Con'] = (data['TIPI3']+data['TIPI8'])/2
data['Es'] =(data['TIPI9']+data['TIPI4'])/2
data['Op'] =(data['TIPI5']+data['TIPI10'])/2

In [146]:
#EDA 결과, 성별에 따라 Emotional Stability/ Conscience/ Open Minded가 투표 여부에 미치는 영향이 크다고 판단되어 feature를 추가해주었습니다.
#(심리성향 1위 코드)
for data in dataset:
  data['Es_gender'] = data['Es']*data['gender']
  data['Con_gender'] = data['Con']*data['gender']
  data['Op_gender'] = data['Op']*data['gender']

In [147]:
tpcoms = list(combinations(tps, 2))
for data in dataset:
  for a,b in tpcoms:
    data['tp_%s_dv_%s'%(a,b)] = data[a]/data[b]

VLC

In [148]:
for data in dataset:
#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무)
  data['VLC_score'] = data[Answers3].sum(axis=1)       
  data['VLC_var'] = data[Answers3].var(axis=1)
  data['VLC_mean'] = data[Answers3].mean(axis = 1)    

age

In [149]:
#age로 age_group object 생성
def age_grouping(age):
    age = (age//10.)*10
    if(age >= 70):
        age = '+70s'
    else:
        age = str(int(age))+'s'
    return age
for data in dataset:
  data['age_group'] = data['age'].apply(age_grouping)

In [150]:
#라벨링 ex) 라벨 인코더
from sklearn.preprocessing import LabelEncoder
for data in dataset:
  encoder = LabelEncoder()
  data['age_group'] = encoder.fit_transform(data['age_group'])

country

In [151]:
for data in dataset:

  #결측값는 'others'로 대체, 위에서 최빈값으로 대체했다면, 실행할 필요x
  data['country']=data['country'].fillna(1)
  data.country = data.country.apply(lambda x : 'others' if x == 1 else x)

  #1번 밖에 입력되지않은 나라는 'others'로
  country_vc = data['country'].value_counts()
  data.country = data.country.apply(lambda x : 'others' if country_vc[x] == 1 else x)

print(x_train['country'].value_counts())
print(x_train['country'][66])

USA    7609
GBR    1109
CAN     915
AUS     525
DEU     473
       ... 
ZMB       2
FRO       2
BWA       2
CUW       2
PAN       2
Name: country, Length: 110, dtype: int64
USA


In [152]:
for data in dataset:
  data = pd.get_dummies(data, columns = ['country'])


In [153]:
data.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,country_UKR,country_URY,country_USA,country_VEN,country_VNM,country_VUT,country_ZAF,country_ZMB,country_ZWE,country_others
0,0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,...,0,0,0,0,0,0,0,0,0,0
1,1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,...,0,0,1,0,0,0,0,0,0,0
2,2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
4,4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
#왼손 오른손은 상관없는 피처라고 판단후 삭제
for data in dataset:
  data.drop('hand', axis=1, inplace = True)


In [155]:
for data in dataset:
    print(data['country'])
    data.drop('country', axis=1, inplace = True)

0        USA
1        USA
2        NLD
3        USA
4        ITA
        ... 
14995    USA
14996    USA
14997    USA
14998    USA
14999    BRA
Name: country, Length: 15000, dtype: object
0        CHL
1        USA
2        IDN
3        AUS
4        PHL
        ... 
35447    USA
35448    GBR
35449    USA
35450    CAN
35451    GBR
Name: country, Length: 35452, dtype: object


#모델2


In [156]:
def lgbm_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [158]:
lgbm_archive_4040 = lgbm_rfe_4040(x_train, y_train)

129 0.8354089586452917
116 0.8322078577662424
104 0.8343670881881683
93 0.8351793207154447
83 0.8328607555692962
74 0.835040443071547
66 0.8323409728523019
59 0.8300483392164246
53 0.8314716910024287
47 0.8251570801234684
42 0.8297397542441959


In [159]:
#위의 코드로부터 최적의 피처를 얻어서..
model = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_1 = x_train[lgbm_archive_4040.iloc[0,2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[0,2]])
pred_y1 = pred_y1[:,1]

In [160]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [161]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train, y_train)

129 0.8376274472398306
116 0.8337566251878956
104 0.8355459430636065
93 0.8327562390537877
83 0.8349885768659701
74 0.8363064318909997
66 0.8334213435628226
59 0.8253880877627665
53 0.8235591221884611
47 0.8262100592531982
42 0.819334918475138


In [162]:
model2 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_2 = x_train[lgbm_archive_1234.iloc[0,2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[lgbm_archive_1234.iloc[0,2]])
pred_y2 = pred_y2[:,1]

In [163]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [164]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train, y_train)

129 0.8343727072633896
116 0.832220553499679
104 0.8344693040660354
93 0.8317904677355179
83 0.8273418399850965
74 0.840157590783745
66 0.8287646303907341
59 0.8237697707205891
53 0.8258598265397415
47 0.8244226616099006
42 0.8197817027256398


In [165]:
model3 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_3 = x_train[lgbm_archive_99087.iloc[2,2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[lgbm_archive_99087.iloc[2,2]])
pred_y3 = pred_y3[:,1]

In [166]:
def lgbm_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [167]:
lgbm_archive_42 = lgbm_rfe_42(x_train, y_train)

129 0.8501722303834658
116 0.8487872494255149
104 0.8472828239335762
93 0.8543478961644202
83 0.8453576083002152
74 0.845443378110163
66 0.8403547531441257
59 0.8499272971342523
53 0.8435506859282249
47 0.8462567522138973
42 0.8442972862201876


In [168]:
model4 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_4 = x_train[lgbm_archive_42.iloc[3,2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[lgbm_archive_42.iloc[3,2]])
pred_y4 = pred_y4[:,1]

In [169]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
submission = pd.DataFrame()
submission['index'] = submission.index
submission['nerdiness'] = pred_all
submission['index'] = submission.index
#submission = pd.DataFrame({
    #"index" : index,
 #   "nerdiness" : pred_all
#})

submission.to_csv('/content/drive/My Drive/csv/sw/model2.csv', index=False)


#Ensemble

In [170]:
model1 = pd.read_csv('/content/drive/My Drive/csv/sw/model1.csv', index_col = 'index')
model2 = pd.read_csv('/content/drive/My Drive/csv/sw/model2.csv', index_col='index')

pred_y = (model1)*(0.5) + (model2)*(0.5)

test = pd.read_csv('/content/drive/My Drive/csv/sw/test.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'nerdiness': pred_y['nerdiness']
    })

submission.to_csv('/content/drive/My Drive/csv/sw/kim_combined_model1_2_5_5.csv', index=False)